# **코드 목적**

1. 이 코드는 크롤링한 데이터를 input으로 받아 BERT 감정 분류 모델의 Input에 적합한 형태로 바꾸기 위해 짜여졌습니다.

2. 크롤링한 데이터는 글 단위이며 test를 위해 제외한 나머지인 90%의 데이터를 이용했습니다. (df_90.csv)

3. 간단한 전처리와 kss 패키지를 이용해 글 단위 데이터를 문장 단위로 분류합니다.

4. 사전에 정의한 감성사전을 정제한 이후에 이를 기반으로 문장에서의 감정 분포를 계산합니다.

5. 문장에서 가장 많이 차지하고 있는(max) principle emotion을 label로 출력합니다.

6. BERT model에서는 input으로 tsv 파일을 받기 때문에 이를 tsv 파일로 내보냅니다.

   # **Content**
1. Data Load & Preprocessing
2. Extracting Principle Word in Sentiment Dictionary
3. Extracting Principle Emotion in Text
4. Counting Emotion 
5. Saving Output Data





##  Import Packages & Setting Environment

In [ ]:
# clone 받을 경로 지정
! cd "drive/My Drive/Colab Notebooks/Conference"

In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/Conference/TRAIN SET/9:1 datasets

/content/drive/My Drive/Colab Notebooks/Conference/TRAIN SET/9:1 datasets


In [ ]:
# mecab 다운로드
# ! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
! bash "/content/drive/My Drive/Colab Notebooks/Conference/TRAIN SET/Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh"
# ! bash "/content/Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh"

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 3.6MB 33.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-06-05 08:23:55--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.2, 18.205.93.0, 18.205.93.1, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?S

In [ ]:
#kss 설치
!sudo apt install python3-dev
!pip install kss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  r-cran-plogr
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251587 sha256=f7244e8e7b526b14be6c64dbfcd9b5ad06583819bf8aec671cf9444726565312
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


In [ ]:
import pandas as pd
import MeCab
import sys
import string
import re
import kss
import warnings
warnings.filterwarnings(action='ignore')

#1. Data Load & Preprocessing

In [ ]:
def data_load(filename):
  print("Data Loading...\n")
  data = pd.read_csv(filename)

  #문장에서 white space 삭제
  data['main_text'] = data['main_text'].astype(str)
  data['main_text'] = data['main_text'].apply(lambda x : re.sub('\s{2,}', '', x))
  data['main_text'] = data['main_text'].apply(lambda x : re.sub('\t+', '', x))
  
  # 결측행 'missing'으로 채우기
  data = data.fillna('missing')
  data_len = len(data)
  print('Number of Documents :', len(data))

  # kss 사용해서 문장 나누기
  sen = []
  for i in range(data_len):
      s = data['main_text'][i]
      for sent in kss.split_sentences(s):
          sen.append(sent)

  print("Number of Sentences : ", len(sen))
  print("\nShowing sample data ... ")
  for i in range(10):
    print("[",i,"] ", sen[i])

  return sen

In [ ]:
def data_load(filename):
  print("Data Loading...\n")
  data = pd.read_csv(filename)

  #문장에서 white space 삭제
  data['main_text'] = data['main_text'].astype(str)
  data['main_text'] = data['main_text'].apply(lambda x : re.sub('\s{2,}', '', x))
  data['main_text'] = data['main_text'].apply(lambda x : re.sub('\t+', '', x))
  
  # 결측행 'missing'으로 채우기
  data = data.fillna('missing')
  data_len = len(data)
  print('Number of Documents :', len(data))

  # kss 사용해서 문장 나누기
  sen = []
  for i in range(data_len):
      s = data['main_text'][i]
      for sent in kss.split_sentences(s):
          sen.append(sent)

  print("Number of Sentences : ", len(sen))
  print("\nShowing sample data ... ")
  for i in range(10):
    print("[",i,"] ", sen[i])

  return sen

##2. Extracting Principle Word in Sentiment Dictionary


In [ ]:
#감성사전 정의 및 핵심 단어 추출
happiness = ['반가워', '뿌듯해', '고마워', '사랑스러워', '소중해', '편안해', '설레', '좋아해', 
         '귀여워', '스윗', '아름다워', '행복해',
        '웃겨', '감사해', '든든해', '만족해', '믿음직스러워', '상쾌해', '다행이다',
        '유쾌해', '재밌어', '통쾌해', '즐거워', '후련해', '흐뭇해', '예쁘다']
surprised = ['당황스러워', '놀라워', '신기해', '어이없어', '신나', '대단해', '감동', '강렬',
             '깜짝', '놀랐어', '대박이야', '짜릿해', '흥분', '감격스러워', '얼떨떨하다', '어리둥절',
             '기가 막히다', '뜻밖', '엄청난', '소름끼치는']
anger = ['괴로워', '분하다', '억울해', '창피해', '쪽팔려', '허탈해', '귀찮아',
         '귀찮아', '피가 끓는', '넌더리나는', '미칠 것 같아', '기분 나빠',    
         '불쾌해', '질투나', '짜증나', '괘씸해', '열받아', '화나']
disgust = ['시끄러워', '지저분해', '잔인해', '안좋아해', '나빠', '못마땅해', '약오르다', '원망스러워', '샘나다', 
            '지루해', '미워', '부담스러워', '따분해','야속해', '심술', '하찮아', '한심한', '조롱', '뻔뻔한', '밥맛 떨어져', '구역질 나', 
            '무례해', '씁쓸해', '싫어', '끔찍해', '얄미워', '멍청해', '더러워']
sadness = ['미안해', '실망스러워', '씁쓸해', '고단해', '피곤해', '후회해', '가슴을 앓다', '가여워', '서러워', '섭섭해', '속상해', '측은해', 
       '애처로워', '허무해', '힘들어', '좌절', '지치다', '찡하다', '그리워', '뭉클해', '침울해', '공허해','마음이 무거워', '버려진', 
       '죽고싶다', '암담하다', '가슴 아픈', '슬퍼', '우울해', '슬펐', '불쌍해', '안타까워', '울고싶다']
fear =['주눅들다', '떨리다', '무서워', '조심스러워', '걱정', '간절해', '막막해', '부끄러워', '혼란스러워', '겁난다', 
       '조마조마', '위태위태', '섬뜻한', '두근두근', '애간장',
       '두려워', '불안해', '불편해', '혼란스러워', '초조해']

# 감성사전에 있는 단어들에서 핵심 감정만 추출해주는 함수 
# 반환값은 따로 없고 인자로 받은 feeling 자체를 바꿈.
def mainExtract(feeling): #changing the word into main feature
  for i in range(0,len(feeling)):
    t = MeCab.Tagger()
    parsed=t.parse(feeling[i])
    if parsed.split(',')[4]!='Inflect':
      feeling[i] = parsed.split(',')[3]
    else:
      feeling[i] = parsed.split(',')[-1].split('/')[0]


print("---BEFORE EMOTION---")
print(happiness)
print(surprised)
print(anger)
print(disgust)
print(sadness)
print(fear)

mainExtract(happiness)
mainExtract(surprised)
mainExtract(anger)
mainExtract(disgust)
mainExtract(sadness)
mainExtract(fear)

print("\n---AFTER EMOTION---")
print(happiness)
print(surprised)
print(anger)
print(disgust)
print(sadness)
print(fear)

---BEFORE EMOTION---
['반가워', '뿌듯해', '고마워', '사랑스러워', '소중해', '편안해', '설레', '좋아해', '귀여워', '스윗', '아름다워', '행복해', '웃겨', '감사해', '든든해', '만족해', '믿음직스러워', '상쾌해', '다행이다', '유쾌해', '재밌어', '통쾌해', '즐거워', '후련해', '흐뭇해', '예쁘다']
['당황스러워', '놀라워', '신기해', '어이없어', '신나', '대단해', '감동', '강렬', '깜짝', '놀랐어', '대박이야', '짜릿해', '흥분', '감격스러워', '얼떨떨하다', '어리둥절', '기가 막히다', '뜻밖', '엄청난', '소름끼치는']
['괴로워', '분하다', '억울해', '창피해', '쪽팔려', '허탈해', '귀찮아', '귀찮아', '피가 끓는', '넌더리나는', '미칠 것 같아', '기분 나빠', '불쾌해', '질투나', '짜증나', '괘씸해', '열받아', '화나']
['시끄러워', '지저분해', '잔인해', '안좋아해', '나빠', '못마땅해', '약오르다', '원망스러워', '샘나다', '지루해', '미워', '부담스러워', '따분해', '야속해', '심술', '하찮아', '한심한', '조롱', '뻔뻔한', '밥맛 떨어져', '구역질 나', '무례해', '씁쓸해', '싫어', '끔찍해', '얄미워', '멍청해', '더러워']
['미안해', '실망스러워', '씁쓸해', '고단해', '피곤해', '후회해', '가슴을 앓다', '가여워', '서러워', '섭섭해', '속상해', '측은해', '애처로워', '허무해', '힘들어', '좌절', '지치다', '찡하다', '그리워', '뭉클해', '침울해', '공허해', '마음이 무거워', '버려진', '죽고싶다', '암담하다', '가슴 아픈', '슬퍼', '우울해', '슬펐', '불쌍해', '안타까워', '울고싶다']
['주눅들다', '떨리다', '무서워', '조심스러워', '걱정', '간절해', '막막해', '부끄러

## 3. Extracting Principle Emotion in Text

In [ ]:
# ---------- Parser----------#
def parsing(sentence_set):
  print("\nParsing ...")
  mentioned=[] #최종 결과는 각 문장을 분해한 결과를 담은 이차원 리스트 (행 : 문장 / 열 : 해당 문장의 형태소 분해결과의 리스트)

  for i in range(len(sentence_set)):
    sentence = sentence_set[i] #문장 집합에서 문장 하나를 받음
    deleted_special=re.sub('[@#$%^&*/+a-zA-Z.,?!{}()]', ' ', sentence) #deleted special characters
    t = MeCab.Tagger()
    parsed=t.parse(deleted_special)
    splited=parsed.split('\n')[:-2]

    men=[] #문장 하나에 대한 형태소 분석 결과를 받을 변수
    for j in range(0,len(splited)):
      word_type_origin=splited[j].split(',') 
      if word_type_origin[4] == 'Inflect':
        last=word_type_origin[-1]
        get=last.split('/')[0]
      else:
        get=word_type_origin[3]
      #print(word_type_origin) #checking
      #print(mentioned) #checking
      if get not in men:
        men.append(get)

    #한 문장에 대한 형태소 분석이 끝났다면 men을 mentioned에 append
    if len(men) == 0:
      mentioned.append(['NULL'])
    else:
      mentioned.append(men)
    # print("mentioned : \n", mentioned)
  
  print("\nShowing sample data after parsing ...")
  for n in range(10):
      print("[",n,"] ", mentioned[n])

 #print(mentioned) #checking
  return mentioned


# ---------- Emotion Count ----------#
def count_emotion(mentioned): #calculating the ratio of how much emotiones are mentioned there inside the sentense
  print("\nCounting emotion...")
  emotion = []
  for i in range(len(mentioned)):
    men = mentioned[i] 
    emo_result={}
    emo_result['ang']=sum([i in anger for i in men])/len(men)
    emo_result['sur']=sum([i in surprised for i in men])/len(men)
    emo_result['hap']=sum([i in happiness for i in men])/len(men)
    emo_result['dis']=sum([i in disgust for i in men])/len(men)
    emo_result['sad']=sum([i in sadness for i in men])/len(men)
    emo_result['fea']=sum([i in fear for i in men])/len(men)
    emotion.append(emo_result)
  return emotion

## 4. Counting Emotion

In [ ]:
def uwek_emotion(mentioned):
  dic=count_emotion(mentioned)
  max_emotion = []
  for i in range(len(dic)):
    max_value = max(dic[i].values())
    if max_value == 0:
      max_keys = []
    else:
      max_keys = [k for k, v in dic[i].items() if v == max_value] 
    max_emotion.append(max_keys)

  print("\nShowing sample data counting emotion ...")
  for n in range(10):
      print("[",n,"] ", max_emotion[n])
  # print(max_emotion)
  return max_emotion

# 5. Saving Output Data

In [ ]:
def save(brunch, emotion):
  print("\nSaving...")
  result = pd.DataFrame()
  result['text'] = brunch
  result['emotion'] = emotion
  result['len'] = list(map(len, result['emotion']))

  #single_emotion
  result_single = result[result['len'] == 1]
  result_single.reset_index(inplace=True, drop=True)
  result_single['emotion'] = result_single['emotion'].apply(lambda x: x[0])
  result_single.emotion = pd.Categorical(result_single.emotion)
  result_single['label'] = result_single.emotion.cat.codes

  #multi_emotion
  result_multi = result[result['len'] >= 2]
  result_multi.reset_index(inplace=True, drop=True)

  #train test split and output to_tsv
  # train_ratio = 0.8
  # data_len = len(result_single)
  # train = result_single[:int(train_ratio * data_len)][['text', 'label']]
  # test = result_single[int(train_ratio * data_len):][['text', 'label']]
  # train.to_csv('train.txt', sep='\t', index_label='idx')
  # test.to_csv('test.txt', sep='\t', index_label='idx')

  result = result_single[['text', 'label']] 
  result.to_csv('train.txt', sep='\t', index_label='idx')
  print("\n****All processes done!****\n")
  print("train length : ", len(result))


# **Let's start all processes !!!**

In [ ]:
cd ..

/content/drive/My Drive/Colab Notebooks/Conference/TRAIN SET


In [ ]:
#1. Data Load & Preprocessing
brunch = data_load("9:1 datasets/df_90.csv")
#3. Extracting Principle Emotion in Text
mentioned = parsing(brunch)
#4. Counting Emotion
emotion = uwek_emotion(mentioned)
#5. Saving Output Data
save(brunch, emotion)

Data Loading...

Number of Documents : 22626
Number of Sentences :  835255

Showing sample data ... 
[ 0 ]  결혼 준비를 하며 커플들은 사상 초유의 전쟁을 벌인다고 하지만 우리에겐 그런 난관이 존재하지 않았다.
[ 1 ]  계곡물 흐르듯 시원하게 속전속결. 그러나 남편과 결혼을 해 아이를 낳고 한 공간에서 살을 부비고 사니 어쩔 수 없이 부딪히는 부분이 생긴다.
[ 2 ]  신혼 초에는 그렇지 않았다.
[ 3 ]  우리는 치약을 어디서부터 짜는지 양말을 왜 뒤집어 놓는가 같은 사소한 문제를 두고 다툰다고 말하는 부부들을 보며 의아해했다.
[ 4 ]  아니 그게 싸울 거리가 되는 건가 치약을 내가 밑에서부터 짠다면 너는 그냥 위에서부터 짜면 될 거 아냐 어차피 치약은 손으로 조물조물거리면 모양이 바뀌는데하지만 아이가 태어나자 그간의 우리 사이를 흐르던 인내심들이 조금씩 바닥을 드러냈다.
[ 5 ]  아무래도 우리가 가진 인내심을 거의 아이에게 쏟아부었기 때문일 것이다.
[ 6 ]  그렇다고 우리가 언성을 높이며 싸운 적은 없다.
[ 7 ]  우리의 싸움은 보통 나의 무표정에서 시작된다.
[ 8 ]  딱딱하게 굳은 얼굴로 집안 주위를 쓱 둘러보다 수다맨이 지하철 노선을 읊어대듯 잔소리를 해대는 것이다.
[ 9 ]  그러면 남편은 핸드폰을 들고 조용히 안방으로 가 가구처럼 몸을 쭈그리고 내 기분이 풀릴 때까지 방에서 나오지 않는다.

Parsing ...

Showing sample data after parsing ...
[ 0 ]  ['결혼', '준비', '를', '하', '며', '커플', '들', '은', '사상', '초유', '의', '전쟁', '을', '벌이', '지만', '우리', '에게', '그런', '난관', '이', '존재', '지', '않', '았', '다']
[ 1 ]  ['계곡물', '흐르', '듯', '시원', '하', '게', '속전속결', '그러나', '